## **1. 라이브러리 불러오기**

In [ ]:
import os

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('seaborn-whitegrid')

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX


import seaborn as sns

%matplotlib inline

import itertools

In [ ]:
sm.__version__

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/data/')

## **2. 데이터 불러오기**

In [ ]:
elec_df = pd.read_csv('Electric_Production.csv')
elec_df.head(10)

In [ ]:
elec_df.info()

In [ ]:
elec_df.columns=['Date', 'Consumption']
elec_df = elec_df.dropna()
elec_df['Date'] = pd.to_datetime(elec_df['Date'])
elec_df.set_index('Date', inplace=True) #set date as index

In [ ]:
elec_df.head(20)

In [ ]:
elec_df.info()

In [ ]:
fig = elec_df.plot()

In [ ]:
# Seasonal decomposition plot : Seasonal decomposition using moving averages.
# https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.seasonal_decompose.html

#sm.__version__ '0.10.1'
#decompostion = sm.tsa.seasonal_decompose(elec_df, model = 'additive', freq = 12)
#sm.__version__ '0.13.0'
decompostion = sm.tsa.seasonal_decompose(elec_df, model = 'additive', period =1)

fig = decompostion.plot()
fig.set_size_inches(10,10)
plt.show()

# 아래 그래프를 데이터로 추출 https://dodonam.tistory.com/89
# decompostion.observed
# decompostion.trend
# decompostion.seasonal
# decompostion.resid

In [ ]:
train_data, test_data = train_test_split(elec_df, test_size = 0.3, shuffle = False)

In [ ]:
# ACF, PACF

import time
start = time.time()  # 시작 시간 저장

fig, ax = plt.subplots(1, 2, figsize=(10,5))
fig.suptitle('Raw data')
sm.graphics.tsa.plot_acf(elec_df, lags = 30, ax = ax[0])
sm.graphics.tsa.plot_pacf(elec_df, lags = 30, ax = ax[1])
print("time :", time.time() - start)

In [ ]:
# Dickey-Fuller test
# p-value가 0.05보다 크면 비정상성

result = adfuller(elec_df["Consumption"])
print('p-value : ', result[1])

print('-------------------------------------------------------------------------------------------------------')

dfoutput = pd.Series(result[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in result[4].items(): 
    dfoutput['Critical Value (%s)'%key] = value
pvalue = result[1]
if pvalue < 0.05:
    print('p-value = %.4f. The series is likely stationary.' % pvalue)
else:
    print('p-value = %.4f. The series is likely non-stationary.' % pvalue)
    
print(dfoutput)

In [ ]:
# Differencing 차분

diff_train_data = train_data.copy()
diff_train_data = diff_train_data['Consumption'].diff() #default가 1차 차분
diff_train_data = diff_train_data.dropna() # 차분 하면 마지막 값이 결측치
print('------ Raw data -----')
print(train_data)
print('------ Differenced data -----')
print(diff_train_data)

In [ ]:
# Differenced data plot

plt.figure(figsize = (12,8))
plt.subplot(211)
plt.plot(train_data['Consumption'])
plt.legend(['Raw data (Nonstationary)'])

plt.subplot(212)
plt.plot(diff_train_data, 'orange')
plt.legend(['Differenced data (Stationary)'])

plt.show()

In [ ]:
# ACF, PACF

fig, ax = plt.subplots(1, 2, figsize=(10,5))
fig.suptitle('Differenced data')
sm.graphics.tsa.plot_acf(diff_train_data, lags = 50, ax = ax[0])
sm.graphics.tsa.plot_pacf(diff_train_data, lags = 50, ax = ax[1])

In [ ]:
# Dickey-Fuller test
# p-value가 0.05보다 크면 비정상성

result = adfuller(diff_train_data)
print('p-value : ', result[1])

print('-------------------------------------------------------------------------------------------------------')

dfoutput = pd.Series(result[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in result[4].items(): 
    dfoutput['Critical Value (%s)'%key] = value
pvalue = result[1]
if pvalue < 0.05:
    print('p-value = %.4f. The series is likely stationary.' % pvalue)
else:
    print('p-value = %.4f. The series is likely non-stationary.' % pvalue)
    
print(dfoutput)

In [ ]:
# ARIMA model fitiing 
# The (p, d, q) order of the model for the number of AR parameters, differences, and MA parameters to use.

# sm.__version__ '0.13.0'
#from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(train_data, order=(1,1,0))
model_fit = model.fit()
model_fit.summary()

In [ ]:
# 0.10.2
#model_fit.plot_predict()

# 0.13.1
plt.plot(train_data)
plt.plot(model_fit.fittedvalues, color='red')
print('RSS : %f' %sum((model_fit.fittedvalues-diff_train_data)**2))

In [ ]:
# Parameter search

print('Examples of parameter combinations for Seadonal ARIMA')
p = range(0,3)
d = range(1,2)
q = range(0,3)
pdq = list(itertools.product(p, d, q))

aic = []
for i in pdq:
    model = ARIMA(train_data, order=(i))
    model_fit = model.fit()
    print('ARIMA:', i, '>> AIC :', round(model_fit.aic,2))
    aic.append(round(model_fit.aic,2))

# 에러 
# The computed initial MA coefficients are not invertible
# You should induce invertibility, choose a different model order, or you can
# pass your own start_params.
# SARIMA 쓰던지 12버전 이후의 statsmodels.tsa.arima.model.ARIMA 사용

In [ ]:
model_opt = ARIMA(train_data.values, order=(2, 1, 2))

model_opt_fit = model_opt.fit()
model_opt_fit.summary()

In [ ]:
# forcast 대신 get_forcast

prediction = model_opt_fit.get_forecast(len(test_data))
predicted_value = prediction.predicted_mean
print(prediction.conf_int())
predicted_ub = prediction.conf_int()[:][0]
predicted_lb = prediction.conf_int()[:][1]
predict_index = list(test_data.index)
r2 = r2_score(test_data, predicted_value)

In [ ]:
fig, ax= plt.subplots(figsize=(12,6))
ax.plot(elec_df.index, elec_df)
ax.vlines(pd.to_datetime('2008-08-01'), 0,200, linestyle ='--', color='r',
         label = 'Start of  Forcast')

ax.plot(predict_index, predicted_value, label='prediction')
ax.fill_between(predict_index, predicted_lb, predicted_ub, color='k', alpha= 0.1, label='0.95 prediction interval')
ax.legend(loc='upper left')
plt.suptitle(f'ARIMA {optimal[0][0][0]} prediction results (r2score: {round(r2,2)})')
plt.show()

In [ ]:
print('Examples of parameter combinations of SARIMA...')
p = range(0,3)
d = range(1,2)
q = range(0,3)

pdq = list(itertools.product(p,d,q))

seasonal_pdq = [(x[0], x[1], x[2], 12) for x in pdq]
print(pdq)
print(seasonal_pdq)

aic = []
params = []
for i in pdq:
    for j in seasonal_pdq:
        try: 
            model = SARIMAX(train_data.values, order=(i), seasonal_order=(j))
            model_fit = model.fit()
            print(f'SARIMA parameter : {i} {j} >> AIC: {round(model_fit.aic,2)}')
            params.append((i,j))
            aic.append(round(model_fit.aic,2))
        except:
            continue
        # try, except 구문 , try 중 에러 나면 except 수행

In [ ]:
print(min(aic))
# ctrl+F SARIMA parameter : (1, 1, 0) (1, 1, 2, 12) >> AIC: 751.15

optimal = [(params[i], j) for i, j in enumerate(aic) if j==min(aic)]
optimal

In [ ]:
#model_opt = SARIMAX(train_data, order = (1,1,0), seasonal_order = (1,1,2,12))

model_opt = SARIMAX(train_data, order = optimal[0][0][0], seasonal_order = optimal[0][0][1])
model_opt_fit = model_opt.fit()
model_opt_fit.summary()

In [ ]:
# ARIMA와 코드가 살짝 다름(forcast 대신 get_forcast)

prediction = model_opt_fit.get_forecast(len(test_data))
predicted_value = prediction.predicted_mean
predicted_ub = prediction.conf_int().iloc[:,0]
predicted_lb = prediction.conf_int().iloc[:,1]
predict_index = list(test_data.index)
r2 = r2_score(test_data, predicted_value)

In [ ]:
fig, ax= plt.subplots(figsize=(12,6))
ax.plot(elec_df.index, elec_df)
ax.vlines(pd.to_datetime('2008-08-01'), 0,200, linestyle ='--', color='r',
         label = 'Start of  Forcast')

ax.plot(predict_index, predicted_value, label='prediction')
ax.fill_between(predict_index, predicted_lb, predicted_ub, color='k', alpha= 0.1, label='0.95 prediction interval')
ax.legend(loc='upper left')
plt.suptitle(f'SARIMA {optimal[0][0][0], optimal[0][0][1]} prediction results (r2score: {round(r2,2)})')
plt.show()